In [ ]:
# Install all required libraries
!pip install unbabel-comet
!pip install -q transformers accelerate evaluate bert_score rouge_score sacremoses nltk psutil pandas sacrebleu sentencepiece

from huggingface_hub import login

# Paste your token here
login(token='hugging face token goes here')

# Download NLTK data needed for BLEU and ROUGE
import nltk
nltk.download('punkt')

# Imports
import time
import torch
import psutil
import logging
from nltk.translate.bleu_score import sentence_bleu
from transformers import T5ForConditionalGeneration, T5Tokenizer
import evaluate

# Suppress unwanted warnings
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)

# Detect device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Example sentences per language
paragraph_data = [
    {"code": "ar", "lang": "Arabic", "src": "أنا أتقدم للحصول على تأشيرة سياحية لزيارة الولايات المتحدة. أنوي البقاء لمدة أسبوعين وزيارة معالم مشهورة مثل تمثال الحرية وساحة التايمز. سأعود إلى بلدي بعد انتهاء عطلتي.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"code": "es", "lang": "Spanish", "src": "Estoy solicitando una visa de turista para visitar los Estados Unidos. Tengo la intención de quedarme dos semanas y visitar lugares famosos como la Estatua de la Libertad y Times Square. Regresaré a mi país después de mis vacaciones.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"code": "fr", "lang": "French", "src": "Je demande un visa de touriste pour visiter les États-Unis. J’ai l’intention de rester deux semaines et de visiter des sites célèbres comme la Statue de la Liberté et Times Square. Je retournerai dans mon pays après mes vacances.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"code": "ru", "lang": "Russian", "src": "Я подаю заявку на туристическую визу для поездки в Соединенные Штаты. Я собираюсь остаться на две недели и посетить известные достопримечательности, такие как Статуя Свободы и Таймс-сквер. Я вернусь в свою страну после отпуска.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"code": "ro", "lang": "Romanian", "src": "Solicit o viză turistică pentru a vizita Statele Unite. Intenționez să stau două săptămâni și să vizitez repere populare precum Statuia Libertății și Times Square. Mă voi întoarce în țara mea după vacanță.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"code": "bg", "lang": "Bulgarian", "src": "Кандидатствам за туристическа виза за посещение на Съединените щати. Възнамерявам да остана две седмици и да посетя популярни забележителности като Статуята на свободата и Таймс Скуеър. Ще се върна в родината си след ваканцията.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"code": "cs", "lang": "Czech", "src": "Žádám o turistické vízum pro návštěvu Spojených států. Mám v úmyslu zůstat dva týdny a navštívit známá místa jako Sochu svobody a Times Square. Po dovolené se vrátím do své domovské země.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"code": "da", "lang": "Danish", "src": "Jeg ansøger om et turistvisum for at besøge USA. Jeg har til hensigt at blive i to uger og besøge kendte vartegn som Frihedsgudinden og Times Square. Jeg vender tilbage til mit hjemland efter ferien.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"code": "de", "lang": "German", "src": "Ich beantrage ein Touristenvisum, um die Vereinigten Staaten zu besuchen. Ich beabsichtige, zwei Wochen zu bleiben und bekannte Sehenswürdigkeiten wie die Freiheitsstatue und den Times Square zu besuchen. Nach meinem Urlaub kehre ich in mein Heimatland zurück.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"code": "et", "lang": "Estonian", "src": "Taotlen turismiviisat Ameerika Ühendriikide külastamiseks. Kavatsen jääda kaheks nädalaks ja külastada kuulsaid vaatamisväärsusi nagu Vabadussammas ja Times Square. Pärast puhkust naasen oma koduriiki.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
]

# mBART language code map for T5-compatible format
LANG_CODE_MAP = {
    "ar": "ar", "es": "es", "fr": "fr", "ru": "ru", "ro": "ro",
    "bg": "bg", "cs": "cs", "da": "da", "de": "de", "et": "et",
}

# Load evaluation metrics
bleu_mod      = evaluate.load("bleu")
bertscore_mod = evaluate.load("bertscore")
meteor_mod    = evaluate.load("meteor")
rouge_mod     = evaluate.load("rouge")
ter_mod       = evaluate.load("ter")
comet_mod     = evaluate.load("comet")

# Helpers to measure system resources
def measure_system():
    proc = psutil.Process()
    return proc.memory_info().rss / 1024**2, psutil.cpu_percent(interval=None)

# Translate & time function for T5
def translate_and_time(model, tokenizer, text: str, language_code: str):
    input_text = f"translate {language_code} to en: {text}"
    t0 = time.perf_counter()
    inputs = tokenizer(input_text, return_tensors="pt", padding=True).to(device)
    outputs = model.generate(**inputs)
    t1 = time.perf_counter()
    pred = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    tokens = len(tokenizer.tokenize(text))
    return pred, t1 - t0, t1 - t0, tokens  # using same for first-token and total

# Load T5 model and tokenizer
model_name = "google-t5/t5-base"
tokenizer  = T5Tokenizer.from_pretrained(model_name)
model      = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
model.eval()

# Evaluate each language
for item in paragraph_data:
    code, lang, src, ref = item["code"], item["lang"], item["src"], item["ref"]
    print(f"\n=== {lang} → English ===")

    # Measure before
    ram_before, cpu_before = measure_system()

    # Translate
    t5_code = LANG_CODE_MAP[code]
    pred, t_first, t_total, token_count = translate_and_time(model, tokenizer, src, t5_code)

    # Measure after
    ram_after, cpu_after = measure_system()

    # Compute metrics
    bleu_score   = bleu_mod.compute(predictions=[pred], references=[[ref]])["bleu"]
    bert_res     = bertscore_mod.compute(predictions=[pred], references=[ref], lang="en")
    meteor_score = meteor_mod.compute(predictions=[pred], references=[[ref]])["meteor"]
    rouge_score  = rouge_mod.compute(predictions=[pred], references=[ref])["rougeL"]
    ter_score    = ter_mod.compute(predictions=[pred], references=[[ref]])["score"]
    comet_res    = comet_mod.compute(sources=[src], predictions=[pred], references=[ref])
    comet_score  = next(v for v in comet_res.values() if isinstance(v, (int, float)))

    # Print
    print(f"Source    : {src}")
    print(f"Reference : {ref}")
    print(f"Predicted : {pred}\n")
    print(f"BLEU      : {bleu_score:.4f}")
    print(f"BERTScore : {bert_res['f1'][0]:.4f}")
    print(f"METEOR    : {meteor_score:.4f}")
    print(f"ROUGE-L   : {rouge_score:.4f}")
    print(f"TER       : {ter_score:.4f}")
    print(f"COMET     : {comet_score:.4f}\n")
    print(f"Time to 1st tok : {t_first:.4f} s")
    print(f"Total inf time  : {t_total:.4f} s")
    print(f"Tokens/sec      : {token_count / t_total:.2f}")
    if device.type=="cuda":
        print(f"VRAM  : {(torch.cuda.max_memory_allocated() / 1024**2):.2f} MB")
    print(f"RAM   : {ram_after-ram_before:.2f} MB")
    print(f"CPU   : {cpu_after-cpu_before:.2f} %")
    size_mb = sum(p.numel() for p in model.parameters())*4/1024**2
    print(f"Model size : {size_mb:.2f} MB")
    print("="*50)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Using device: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

...

Source    : Jeg ansøger om et turistvisum for at besøge USA. Jeg har til hensigt at blive i to uger og besøge kendte vartegn som Frihedsgudinden og Times Square. Jeg vender tilbage til mit hjemland efter ferien.
Reference : I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation.
Predicted : en: Jeg ansger om et turistvisum

BLEU      : 0.0000
BERTScore : 0.7978
METEOR    : 0.0000
ROUGE-L   : 0.0000
TER       : 100.0000
COMET     : 0.4535

Time to 1st tok : 1.9563 s
Total inf time  : 1.9563 s
Tokens/sec      : 54.69
RAM   : 0.00 MB
CPU   : -0.80 %
Model size : 850.31 MB

=== German → English ===


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Source    : Ich beantrage ein Touristenvisum, um die Vereinigten Staaten zu besuchen. Ich beabsichtige, zwei Wochen zu bleiben und bekannte Sehenswürdigkeiten wie die Freiheitsstatue und den Times Square zu besuchen. Nach meinem Urlaub kehre ich in mein Heimatland zurück.
Reference : I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation.
Predicted : en: Ich beantrage ein Touristenvisum, um die Vereinigten Staaten

BLEU      : 0.0000
BERTScore : 0.8043
METEOR    : 0.0000
ROUGE-L   : 0.0000
TER       : 100.0000
COMET     : 0.4723

Time to 1st tok : 1.8051 s
Total inf time  : 1.8051 s
Tokens/sec      : 38.78
RAM   : 0.00 MB
CPU   : -32.80 %
Model size : 850.31 MB

=== Estonian → English ===


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Source    : Taotlen turismiviisat Ameerika Ühendriikide külastamiseks. Kavatsen jääda kaheks nädalaks ja külastada kuulsaid vaatamisväärsusi nagu Vabadussammas ja Times Square. Pärast puhkust naasen oma koduriiki.
Reference : I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation.
Predicted : et to en: Taotlen turismiviisat Ameer

BLEU      : 0.0000
BERTScore : 0.7892
METEOR    : 0.0124
ROUGE-L   : 0.0426
TER       : 97.5610
COMET     : 0.2851

Time to 1st tok : 1.9327 s
Total inf time  : 1.9327 s
Tokens/sec      : 57.95
RAM   : 0.00 MB
CPU   : -0.70 %
Model size : 850.31 MB
